# Run Training Experiment

## 1. Load and connect to workplace

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 2. Create a parameterised training script

### Training Folder Set Up

In [ ]:
import os, shutil

# Create a folder for the experiment files
training_folder = 'training'
os.makedirs(training_folder, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy(
    '../1. Analysis and Preprocessing/preprocessed_datasets/market_segmentation_interaction.csv',
    os.path.join(training_folder, "market_segmentation_interaction.csv")
)
shutil.copy(
    '../1. Analysis and Preprocessing/preprocessed_datasets/market_segmentation.csv',
    os.path.join(training_folder, "market_segmentation.csv")
)

### Logistic Regression

In [ ]:
%%writefile $training_folder/lg-training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelBinarizer

# Get the experiment run context
run = Run.get_context()

# Set regularization hyperparameter
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# load the market segmentation dataset
print("Loading Data...")
market_segmentation = pd.read_csv('market_segmentation_interaction.csv')

# Separate features and labels
X, y = market_segmentation.drop(columns="Segmentation"), market_segmentation.Segmentation

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg).fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_scores = model.predict_proba(X_test)
for class_of_interest in ["A", "B", "C", "D"]:
    class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
    auc = roc_auc_score(y_onehot_test[:,class_id],y_scores[:,class_id])
    print(f'AUC {class_of_interest} vs rest: ' + str(auc))
    run.log(f'AUC {class_of_interest} vs rest', np.float(auc))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/lg-model.pkl')

run.complete()

### Random Forest

In [ ]:
%%writefile $training_folder/rf-training.py
# Import libraries
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelBinarizer

# Get the experiment run context
run = Run.get_context()

# Set hyperparameters
parser = argparse.ArgumentParser()
parser.add_argument('--max_features', type=int, dest='max_features', default=4)
parser.add_argument('--max_depth', type=int, dest='max_depth', default=None)
parser.add_argument('--min_samples_split', type=int, dest='min_samples_split', default=2)
parser.add_argument('--min_samples_leaf', type=int, dest='min_samples_leaf', default=1)
args = parser.parse_args()
max_features = args.max_features
max_depth = args.max_depth
min_samples_split = args.min_samples_split
min_samples_leaf = args.min_samples_leaf

# load the market segmentation dataset
print("Loading Data...")
market_segmentation = pd.read_csv('market_segmentation.csv')

# Separate features and labels
X, y = market_segmentation.drop(columns="Segmentation"), market_segmentation.Segmentation

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a random forest model
print(
    'Training a random forest model with max features of', max_features,
    'max depth of', max_depth,
    'min samples split of', min_samples_split,
    'and min_samples_leaf of', min_samples_leaf
)
run.log('Maximum depth of tree', max_depth)
run.log('Maximum features per split', max_features)
run.log('Minimum samples requierd for split', min_samples_split)
run.log('Minimum samples per leaf', min_samples_leaf)
model = RandomForestClassifier(
    max_features=max_features,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf
).fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# calculate AUC
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_scores = model.predict_proba(X_test)
for class_of_interest in ["A", "B", "C", "D"]:
    class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
    auc = roc_auc_score(y_onehot_test[:,class_id],y_scores[:,class_id])
    print(f'AUC {class_of_interest} vs rest: ' + str(auc))
    run.log(f'AUC {class_of_interest} vs rest', np.float(auc))

# Save the trained model in the outputs folder
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/rf-model.pkl')

run.complete()

## 3. Create a compute cluster

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aguo921"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

## 4. Create environment

In [ ]:
%%writefile $training_folder/environment.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- numpy
- pip
- pip:
  - azureml-defaults

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.runconfig import DockerConfiguration

# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", training_folder + "/environment.yml")

## 5. Run the script with arguments

### Logistic Regression

In [ ]:
# Create a script config for training the logistic regression model
script_config = ScriptRunConfig(
    source_directory=training_folder,
    script='lg-training.py',
    arguments = ['--reg_rate', 0.1],
    environment=env,
    docker_runtime_config=DockerConfiguration(use_docker=True),
    compute_target=cluster_name
) 

# submit the experiment run for training the logistic regression model
experiment_name = 'train-segmentation-lg'
experiment = Experiment(workspace=ws, name=experiment_name)
run_lg = experiment.submit(config=script_config)

# Block until the experiment run has completed
run_lg.wait_for_completion()

In [ ]:
# Get logged metrics and files for logistic regression training
metrics = run_lg.get_metrics()
for key in metrics.keys():
    print(key, ":", metrics.get(key))
print('\n')
for file in run_lg.get_file_names():
    print(file)

### Random Forest

In [ ]:
# Create a script config for training the random forest model
script_config = ScriptRunConfig(
    source_directory=training_folder,
    script='rf-training.py',
    arguments = ['--max_depth', 50, '--max_features', 5, '--min_samples_split', 4, '--min_samples_leaf', 4],
    environment=env,
    docker_runtime_config=DockerConfiguration(use_docker=True),
    compute_target=cluster_name
) 

# submit the experiment run for training the random forest model
experiment_name = 'train-segmentation-rf'
experiment = Experiment(workspace=ws, name=experiment_name)
run_rf = experiment.submit(config=script_config)

# Block until the experiment run has completed
run_rf.wait_for_completion()

In [ ]:
# Get logged metrics and files for random forest training
metrics = run_rf.get_metrics()
for key in metrics.keys():
    print(key, ":", metrics.get(key))
print('\n')
for file in run_rf.get_file_names():
    print(file)

## 5. Register the model

In [ ]:
from azureml.core import Model

In [ ]:
# Register the logistic regression model
run_lg.register_model(
    model_path='outputs/lg-model.pkl', model_name='segmentation-lg',
    tags={'Training context':'Script'},
    properties={
        'AUC A vs rest': run_lg.get_metrics()['AUC A vs rest'],
        'AUC B vs rest': run_lg.get_metrics()['AUC B vs rest'],
        'AUC C vs rest': run_lg.get_metrics()['AUC C vs rest'],
        'AUC D vs rest': run_lg.get_metrics()['AUC D vs rest'],
        'Accuracy': run_lg.get_metrics()['Accuracy']
    }
)

In [ ]:
# Register the random forest model
run_rf.register_model(
    model_path='outputs/rf-model.pkl', model_name='segmentation-rf',
    tags={'Training context':'Script'},
    properties={
        'AUC A vs rest': run_rf.get_metrics()['AUC A vs rest'],
        'AUC B vs rest': run_rf.get_metrics()['AUC B vs rest'],
        'AUC C vs rest': run_rf.get_metrics()['AUC C vs rest'],
        'AUC D vs rest': run_rf.get_metrics()['AUC D vs rest'],
        'Accuracy': run_rf.get_metrics()['Accuracy']
    }
)

In [ ]:
# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')